# Sample Script to update testplan using a template

## Description 
- This script automates the process of applying a predefined test plan template over one or more selected test plans.
- It takes in a list of the selected test plan's IDs, and the user entered test plan template ID as input.
- It can update at most 1000 test plans at a time. 
  
Note: To customize the automate algorithm, modify the following sections:   
- [APIs and Algorithms](#APIs-and-algorithms)
- [Actions and Output](#Actions-and-output) 

## Imports

 Import the necessary Python modules to execute the notebook. **`nisystemlink.clients`** provides the predefined models and methods for `TestPlans` and `TestPlan Templates` APIs. **`scrapbook`** is used to run notebooks and record data for integration with the SystemLink Notebook Execution Service.

In [ ]:
import scrapbook as sb
from typing import List

from nisystemlink.clients.test_plan import TestPlanClient
from nisystemlink.clients.test_plan.models import (
    QueryTestPlansRequest,
    QueryTestPlanTemplatesRequest,
    UpdateTestPlansRequest,
    UpdateTestPlanRequest,
    TestPlan,
    TestPlanTemplate
)

## Parameters

- **`test_plan_ids`**: IDs of the test plans to be updated.
- **`template_id`**: ID of the test plan template used to update the test plans.

Parameters are also listed in the metadata for the **parameters cell**, along with their default values.  
The Notebook Execution services use that metadata to pass parameters to this notebook.  
To view the metadata:
- Select the code cell
- Click the **wrench icon** in the right panel.

### Sample metadata

```json
{
  "papermill": {
    "parameters": {
        "template_id": "",
        "test_plan_ids": []
    }
  },
  "systemlink": {
    "parameters": [
        {
            "display_name": "Test plan IDs",
            "id": "test_plan_ids",
            "type": "string[]"
        },
        {
            "display_name": "Test plan template ID",
            "id": "template_id",
            "type": "string"
        }
    ],
    "version": 1
  },
  "tags": ["parameters"]
}
```
For more information on how parameterization works, review the [papermill documentation](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#how-parameters-work).

In [ ]:
# Add the test plan IDs as a list of strings here
test_plan_ids = []

# Add the test plan template ID here
template_id = "" 

## Python clinet

Initialize TestPlanClient to access test plan and test plan template APIs.

In [ ]:
test_plan_client = TestPlanClient()

## APIs and algorithms

Provides methods to query test plans and test plan templates by their IDs, update test plan details using data from test plan template by invoking the update test plan API.

Retrieve the test plans by its IDs.

In [ ]:
def query_test_plans(test_plan_ids: List[str]) -> List[TestPlan]:
    filter_conditions = ' || '.join(f'id == "{id}"' for id in test_plan_ids)

    request = QueryTestPlansRequest(
        filter=filter_conditions
    )

    response = test_plan_client.query_test_plans(request)
    return response.test_plans

Retrieve the test plan template by its ID.

In [ ]:
def get_test_plan_template(template_id) -> TestPlanTemplate:
    request = QueryTestPlanTemplatesRequest(
        filter = f'id == \"{template_id}\"'
    )

    response = test_plan_client.query_test_plan_templates(request)
    
    if response.test_plan_templates == []:
        print(f"No template found with ID {template_id}")
        return []

    return response.test_plan_templates[0]

Updates the given test plans and returns the IDs of successfully updated test plans.

In [ ]:
def update_test_plans(test_plans) -> List[str]:
    request = UpdateTestPlansRequest(
         test_plans = [UpdateTestPlanRequest(**test_plan.model_dump()) for test_plan in test_plans],
         replace = True
     )

    test_plans = test_plan_client.update_test_plans(request)

    updated_test_plan_ids = [test_plan.id for test_plan in test_plans.updated_test_plans]

    return updated_test_plan_ids

Update the test plans using the given test plan template, and return the IDs of the updated test plans.

In [ ]:
def update_test_plans_with_template(test_plans, test_plan_template) -> List[str]:
    if not test_plans or not test_plan_template:
        return []

    for test_plan in test_plans:
        test_plan.description = test_plan_template.description
        test_plan.test_program = test_plan_template.test_program
        test_plan.estimated_duration_in_seconds = test_plan_template.estimated_duration_in_seconds
        test_plan.system_filter = test_plan_template.system_filter
        test_plan.execution_actions = test_plan_template.execution_actions
        test_plan.properties = test_plan_template.properties
        test_plan.dashboard = test_plan_template.dashboard
    
    updated_test_plan_ids = update_test_plans(test_plans)
    return updated_test_plan_ids

## Actions and output

Validates the inputs, manages the execution flow, and prints the script output and sends the output to the Execution Results page via Scrapbook.

In [ ]:
updated_test_plan_ids = []
failed_test_plan_ids = []

if len(test_plan_ids) == 0:
    print("Required atleast one test plan id")
elif len(test_plan_ids) > 1000:
    print("Update limit exceeded: Only up to 1000 test plans can be updated at a time.")
    sb.glue("Update limit exceeded: Only up to 1000 test plans can be updated at a time.")
elif not template_id:
    print("No template ID provided")
    sb.glue("Failed to fetch template", "No template ID provided")
else:    
    # Fetch test plan template
    test_plan_template = get_test_plan_template(template_id)
    
    if test_plan_template is []:
        sb.glue("Failed to fetch template", template_id)
    else:
        # Fetch test plans
        test_plans = query_test_plans(test_plan_ids)

        # Update test plans with template
        updated_test_plan_ids = update_test_plans_with_template(test_plans, test_plan_template)
        failed_test_plan_ids = list(set(test_plan_ids) - set(updated_test_plan_ids))

        # Output
        print("Total test plans:", len(test_plan_ids))
        print("Test plans updated:", ', '.join(updated_test_plan_ids) if updated_test_plan_ids else "--")
        print("Test plans not updated:", ', '.join(failed_test_plan_ids) if failed_test_plan_ids else "--")
        
        # Executions page output
        sb.glue("Total test plans:", len(test_plan_ids))
        sb.glue("Test plans updated:", ', '.join(updated_test_plan_ids) if updated_test_plan_ids else "--")
        sb.glue("Test plans not updated:", ', '.join(failed_test_plan_ids) if failed_test_plan_ids else "--")

## Next Steps
- Publish this notebook to SystemLink by right-clicking it in the JupyterLab File Browser with the interface as **`Test Plan Automations`**.